In [74]:
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python  sklearn matplotlib
#!pip install mediapipe

In [75]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pandas as pd

In [76]:
video_path='data/videos'
np_data_path='np_data'

# Reading csv file

In [77]:
df = pd.read_csv('features_df.csv')
df.video_id = df.video_id.apply(lambda x:  x.replace('A', ''))
print(df)

            gloss video_id                                                url
0            book    69241       http://aslbricks.org/New/ASL-Videos/book.mp4
1            book    07069  https://signstock.blob.core.windows.net/signsc...
2            book    07068  https://s3-us-west-1.amazonaws.com/files.start...
3            book    07070  https://media.asldeafined.com/vocabulary/14666...
4            book    07099     http://www.aslsearch.com/signs/videos/book.mp4
...           ...      ...                                                ...
11975  wheelchair    63047  https://www.signingsavvy.com/signs/mp4/5/5233.mp4
11976  wheelchair    63050  http://www.aslsearch.com/signs/videos/wheelcha...
11977     whistle    63186  https://media.spreadthesign.com/video/mp4/13/9...
11978     whistle    63188  https://www.signingsavvy.com/signs/mp4/9/9961.mp4
11979     whistle    63190  http://www.aslsearch.com/signs/videos/whistle.mp4

[11980 rows x 3 columns]


In [78]:
filtered_df = df
list_data=list(filtered_df['video_id'])

print(list_data)

['69241', '07069', '07068', '07070', '07099', '07074', '69302', '65539', '17710', '17733', '65540', '17734', '17711', '17712', '17713', '17709', '17720', '17721', '17722', '17723', '17724', '12328', '12312', '12311', '12338', '12313', '12314', '12315', '12316', '12317', '12318', '12319', '12320', '12326', '12327', '05728', '05749', '05750', '05729', '05730', '65167', '05731', '05732', '05733', '05734', '05727', '05739', '05740', '05741', '05742', '05743', '09848', '09869', '09849', '09850', '09851', '65328', '09854', '69345', '24955', '24956', '24941', '24960', '24961', '24962', '65824', '24973', '24943', '24946', '24947', '24940', '24952', '24954', '11310', '11330', '11311', '11313', '11309', '69534', '63242', '63226', '63227', '63228', '63229', '63230', '63231', '66778', '63232', '66779', '63225', '63236', '63237', '08929', '08916', '08917', '08918', '08919', '08920', '08921', '65298', '08924', '65299', '65300', '08915', '08925', '65415', '13647', '13648', '13631', '13632', '13633', 

# Creating folder for data

In [79]:
total_class= df['gloss'].unique()
print(total_class)

['book' 'drink' 'computer' ... 'weigh' 'wheelchair' 'whistle']


In [80]:
for item in total_class:
    folder_path = os.path.join(np_data_path, item)
    os.makedirs(folder_path)


In [81]:
def draw_image_keypoint(annotated_image):
    # Draw face landmarks
    mp_drawing.draw_landmarks(
        annotated_image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION
    )

    # Draw left hand landmarks
    mp_drawing.draw_landmarks(
        annotated_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS
    )

    # Draw right hand landmarks
    mp_drawing.draw_landmarks(
        annotated_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS
    )

    # Draw body landmarks
    mp_drawing.draw_landmarks(
        annotated_image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS
    )

    # Display the image with key points
    cv2.imshow("Image with Keypoints", annotated_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [82]:
for item in list_data:
    class_folder=df[df['video_id'] == item]
    class_folder=list(class_folder['gloss'])  
    folder_path = os.path.join(np_data_path,class_folder[0],str(item) )
    os.makedirs(folder_path)

In [83]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [84]:
cap = cv2.VideoCapture('data/videos/00339.mp4')
nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print(nframe)

61.0


In [85]:

def process_video_frame(frame,path,frame_count):
    mp_drawing = mp.solutions.drawing_utils
    mp_holistic = mp.solutions.holistic

    # Create a Holistic instance
    holistic = mp_holistic.Holistic()


    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image to extract the key points
    results = holistic.process(image_rgb)

    # Draw the key points on the image
    annotated_image = image.copy()

    # Release the resources
    holistic.close()

    keypoints=extract_keypoints(results)
    npy_path = os.path.join(path, str(frame_count))
    np.save(npy_path, keypoints)




In [86]:
print(len(final_results))
print(type(final_results))

1662
<class 'numpy.ndarray'>


In [99]:
length = 1662

# Create an empty ndarray of the desired length
keypoints = np.empty(length)
# Fill the array with null values
keypoints.fill(0)
print(keypoints)
def data_padding(frame_count,path):    
    
    for i in range(frame_count,100):        
        npy_path = os.path.join(path, str(i))
        np.save(npy_path, keypoints)

    

[0. 0. 0. ... 0. 0. 0.]


In [100]:
list_data=list_data[:372]
for item in list_data:
    item_path=str(item)+'.mp4'
    path=os.path.join(video_path,item_path)
    print(path)
    class_folder=df[df['video_id'] == item]
    class_folder=list(class_folder['gloss'])  
    data_path = os.path.join(np_data_path,class_folder[0],str(item) )
    cap = cv2.VideoCapture(path)
    nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    print(nframe)
    frame_count = 0
    
    while cap.isOpened():
        sucess, frame = cap.read()
        if not sucess:
            break       
        
        process_video_frame(frame,data_path,frame_count)
        frame_count+=1
    print(frame_count)
    data_padding(frame_count,data_path)
    

data/videos\69241.mp4
75.0
75
data/videos\07069.mp4
30.0
30
data/videos\07068.mp4
68.0
68
data/videos\07070.mp4
86.0
86
data/videos\07099.mp4
87.0
87
data/videos\07074.mp4
41.0
38
data/videos\69302.mp4
77.0
77
data/videos\65539.mp4
44.0
44
data/videos\17710.mp4
70.0
70
data/videos\17733.mp4
93.0
93
data/videos\65540.mp4
46.0
46
data/videos\17734.mp4
89.0
89
data/videos\17711.mp4
81.0
81
data/videos\17712.mp4
43.0
43
data/videos\17713.mp4
91.0
91
data/videos\17709.mp4
105.0
105
data/videos\17720.mp4
41.0
40
data/videos\17721.mp4
49.0
48
data/videos\17722.mp4
49.0
48
data/videos\17723.mp4
60.0
59
data/videos\17724.mp4
30.0
29
data/videos\12328.mp4
88.0
88
data/videos\12312.mp4
101.0
101
data/videos\12311.mp4
72.0
72
data/videos\12338.mp4
107.0
107
data/videos\12313.mp4
81.0
81
data/videos\12314.mp4
43.0
43
data/videos\12315.mp4
57.0
57
data/videos\12316.mp4
43.0
43
data/videos\12317.mp4
53.0
53
data/videos\12318.mp4
45.0
45
data/videos\12319.mp4
40.0
40
data/videos\12320.mp4
102.0
102
da

53
data/videos\33269.mp4
72.0
72
data/videos\33270.mp4
78.0
78
data/videos\33273.mp4
38.0
36
data/videos\33274.mp4
30.0
30
data/videos\69396.mp4
89.0
89
data/videos\34823.mp4
71.0
71
data/videos\34824.mp4
67.0
67
data/videos\34825.mp4
42.0
42
data/videos\34826.mp4
35.0
35
data/videos\34827.mp4
81.0
81
data/videos\34830.mp4
32.0
29
data/videos\34831.mp4
49.0
46
data/videos\34832.mp4
42.0
41
data/videos\34839.mp4
86.0
86
data/videos\69402.mp4
82.0
82
data/videos\36946.mp4
94.0
94
data/videos\36930.mp4
68.0
68
data/videos\36931.mp4
82.0
82
data/videos\36932.mp4
75.0
75
data/videos\36933.mp4
74.0
74
data/videos\36936.mp4
39.0
38
data/videos\36937.mp4
46.0
45
data/videos\66147.mp4
66.0
66
data/videos\66146.mp4
61.0
61
data/videos\36929.mp4
61.0
61
data/videos\69413.mp4
60.0
60
data/videos\38994.mp4
78.0
78
data/videos\38999.mp4
31.0
30
data/videos\39000.mp4
34.0
33
data/videos\38990.mp4
78.0
78
data/videos\39006.mp4
81.0
81
data/videos\38991.mp4
78.0
78
data/videos\38995.mp4
83.0
83
data/vi

In [101]:

position = list_data.index('06839')
print("Position:", position)

ValueError: '06839' is not in list

In [90]:
class_folder=df[df['video_id'] == '06839']
print(class_folder)

    gloss video_id                                                url
372  blue    06839  https://www.signingsavvy.com/signs/mp4/22/2214...


In [93]:
new_df=df[:373]

In [95]:
data_class=new_df['gloss'].unique()
print(data_class)

['book' 'drink' 'computer' 'before' 'chair' 'go' 'clothes' 'who' 'candy'
 'cousin' 'deaf' 'fine' 'help' 'no' 'thin' 'walk' 'year' 'yes' 'all'
 'black' 'cool' 'finish' 'hot' 'like' 'many' 'mother' 'now' 'orange'
 'table' 'thanksgiving' 'what' 'woman' 'bed' 'blue']


In [97]:
directory_path = np_data_path
import shutil
# Get a list of all directories in the parent directory
all_directories = os.listdir(directory_path)

# Iterate over the directories and remove the ones not in the list
for directory in all_directories:
    if directory not in data_class:
        directory_to_remove = os.path.join(directory_path, directory)
        shutil.rmtree(directory_to_remove)
        print("Removed directory:", directory_to_remove)

Removed directory: np_data\a
Removed directory: np_data\a lot
Removed directory: np_data\abdomen
Removed directory: np_data\able
Removed directory: np_data\about
Removed directory: np_data\above
Removed directory: np_data\accent
Removed directory: np_data\accept
Removed directory: np_data\accident
Removed directory: np_data\accomplish
Removed directory: np_data\accountant
Removed directory: np_data\across
Removed directory: np_data\act
Removed directory: np_data\action
Removed directory: np_data\active
Removed directory: np_data\activity
Removed directory: np_data\actor
Removed directory: np_data\adapt
Removed directory: np_data\add
Removed directory: np_data\address
Removed directory: np_data\adjective
Removed directory: np_data\adjust
Removed directory: np_data\admire
Removed directory: np_data\admit
Removed directory: np_data\adopt
Removed directory: np_data\adult
Removed directory: np_data\advanced
Removed directory: np_data\advantage
Removed directory: np_data\adverb
Removed direc

Removed directory: np_data\bye
Removed directory: np_data\cabbage
Removed directory: np_data\cabinet
Removed directory: np_data\cafeteria
Removed directory: np_data\cake
Removed directory: np_data\calculate
Removed directory: np_data\calculator
Removed directory: np_data\calculus
Removed directory: np_data\california
Removed directory: np_data\call
Removed directory: np_data\calm
Removed directory: np_data\camel
Removed directory: np_data\camera
Removed directory: np_data\camp
Removed directory: np_data\camping
Removed directory: np_data\can
Removed directory: np_data\canada
Removed directory: np_data\cancel
Removed directory: np_data\candidate
Removed directory: np_data\candle
Removed directory: np_data\cannot
Removed directory: np_data\canoe
Removed directory: np_data\captain
Removed directory: np_data\caption
Removed directory: np_data\car
Removed directory: np_data\card
Removed directory: np_data\cards
Removed directory: np_data\care
Removed directory: np_data\careful
Removed direc

Removed directory: np_data\doll
Removed directory: np_data\dollar
Removed directory: np_data\dolphin
Removed directory: np_data\don't want
Removed directory: np_data\done
Removed directory: np_data\door
Removed directory: np_data\dorm
Removed directory: np_data\dormitory
Removed directory: np_data\double
Removed directory: np_data\doubt
Removed directory: np_data\down
Removed directory: np_data\downstairs
Removed directory: np_data\drag
Removed directory: np_data\dragon
Removed directory: np_data\drama
Removed directory: np_data\draw
Removed directory: np_data\drawer
Removed directory: np_data\dream
Removed directory: np_data\dress
Removed directory: np_data\drive
Removed directory: np_data\drop
Removed directory: np_data\drug
Removed directory: np_data\drum
Removed directory: np_data\drunk
Removed directory: np_data\dry
Removed directory: np_data\duck
Removed directory: np_data\due
Removed directory: np_data\dull
Removed directory: np_data\dumb
Removed directory: np_data\during
Remove

Removed directory: np_data\head
Removed directory: np_data\headache
Removed directory: np_data\health
Removed directory: np_data\heap
Removed directory: np_data\hear
Removed directory: np_data\hearing
Removed directory: np_data\hearing aid
Removed directory: np_data\heart
Removed directory: np_data\heart attack
Removed directory: np_data\heaven
Removed directory: np_data\heavy
Removed directory: np_data\helicopter
Removed directory: np_data\hello
Removed directory: np_data\helmet
Removed directory: np_data\her
Removed directory: np_data\here
Removed directory: np_data\herself
Removed directory: np_data\hide
Removed directory: np_data\high
Removed directory: np_data\high school
Removed directory: np_data\highway
Removed directory: np_data\hill
Removed directory: np_data\hippopotamus
Removed directory: np_data\his
Removed directory: np_data\history
Removed directory: np_data\hit
Removed directory: np_data\hockey
Removed directory: np_data\hold
Removed directory: np_data\holy
Removed dire

Removed directory: np_data\necessary
Removed directory: np_data\neck
Removed directory: np_data\necklace
Removed directory: np_data\need
Removed directory: np_data\negative
Removed directory: np_data\negotiate
Removed directory: np_data\neighbor
Removed directory: np_data\nephew
Removed directory: np_data\nervous
Removed directory: np_data\network
Removed directory: np_data\neutral
Removed directory: np_data\never
Removed directory: np_data\new
Removed directory: np_data\new york
Removed directory: np_data\newspaper
Removed directory: np_data\next
Removed directory: np_data\nice
Removed directory: np_data\nickel
Removed directory: np_data\niece
Removed directory: np_data\night
Removed directory: np_data\nine
Removed directory: np_data\nineteen
Removed directory: np_data\none
Removed directory: np_data\noon
Removed directory: np_data\normal
Removed directory: np_data\north
Removed directory: np_data\northwest
Removed directory: np_data\nose
Removed directory: np_data\not
Removed directo

Removed directory: np_data\realize
Removed directory: np_data\really
Removed directory: np_data\reason
Removed directory: np_data\receive
Removed directory: np_data\recent
Removed directory: np_data\recliner
Removed directory: np_data\recognize
Removed directory: np_data\recommend
Removed directory: np_data\recover
Removed directory: np_data\red
Removed directory: np_data\reduce
Removed directory: np_data\refer
Removed directory: np_data\referee
Removed directory: np_data\refuse
Removed directory: np_data\regular
Removed directory: np_data\rehearse
Removed directory: np_data\reject
Removed directory: np_data\relate
Removed directory: np_data\relationship
Removed directory: np_data\relax
Removed directory: np_data\release
Removed directory: np_data\relief
Removed directory: np_data\religion
Removed directory: np_data\rely
Removed directory: np_data\remember
Removed directory: np_data\remote control
Removed directory: np_data\remove
Removed directory: np_data\rent
Removed directory: np_d

Removed directory: np_data\start
Removed directory: np_data\statistics
Removed directory: np_data\stay
Removed directory: np_data\steal
Removed directory: np_data\steel
Removed directory: np_data\stepfather
Removed directory: np_data\sticky
Removed directory: np_data\still
Removed directory: np_data\sting
Removed directory: np_data\stink
Removed directory: np_data\stir
Removed directory: np_data\stitch
Removed directory: np_data\stomach
Removed directory: np_data\stop
Removed directory: np_data\store
Removed directory: np_data\story
Removed directory: np_data\straight
Removed directory: np_data\strange
Removed directory: np_data\straw
Removed directory: np_data\strawberry
Removed directory: np_data\street
Removed directory: np_data\stress
Removed directory: np_data\stretch
Removed directory: np_data\strict
Removed directory: np_data\strong
Removed directory: np_data\structure
Removed directory: np_data\struggle
Removed directory: np_data\stubborn
Removed directory: np_data\stuck
Remove

Removed directory: np_data\weekly
Removed directory: np_data\weigh
Removed directory: np_data\weight
Removed directory: np_data\weird
Removed directory: np_data\welcome
Removed directory: np_data\west
Removed directory: np_data\wet
Removed directory: np_data\whale
Removed directory: np_data\whatever
Removed directory: np_data\wheelchair
Removed directory: np_data\when
Removed directory: np_data\where
Removed directory: np_data\which
Removed directory: np_data\while
Removed directory: np_data\whistle
Removed directory: np_data\white
Removed directory: np_data\why
Removed directory: np_data\wide
Removed directory: np_data\wife
Removed directory: np_data\will
Removed directory: np_data\willing
Removed directory: np_data\win
Removed directory: np_data\wind
Removed directory: np_data\window
Removed directory: np_data\wine
Removed directory: np_data\winter
Removed directory: np_data\wish
Removed directory: np_data\with
Removed directory: np_data\within
Removed directory: np_data\without
Remo